# Loading packages

In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from pandas.api.types import CategoricalDtype

# Loading data

In [6]:
data= pd.read_pickle("../data/modified_exclusions/pt_replication_modified_exclusions_data.pkl")

# Create original PT proportions

In [7]:
original_pt_proportions = pd.Series([0.18, 0.83, 0.20, 0.65, 0.14, 0.73, 0.92, 0.42, 0.92, 0.30, 0.22, 0.16, 0.69, 0.18, 0.70, 0.72, 0.17],
                                   index=[str(i) for i in np.arange(1, 18)])
original_pt_proportions

1     0.18
2     0.83
3     0.20
4     0.65
5     0.14
6     0.73
7     0.92
8     0.42
9     0.92
10    0.30
11    0.22
12    0.16
13    0.69
14    0.18
15    0.70
16    0.72
17    0.17
dtype: float64

# Preparing long format data

In [8]:
long_data = data.groupby("Country")[[str(i) for i in np.arange(1, 18)]].mean().reset_index().melt(id_vars="Country", value_vars=[str(i) for i in np.arange(1, 18)],
                                                                                     var_name="Item", value_name="Proportion")

In [9]:
long_data2 = pd.DataFrame(original_pt_proportions)
long_data2 = long_data2.reset_index()
long_data2.columns = ["Item", "Proportion"]
long_data2["Item"] = long_data2["Item"].astype(str)
long_data2["Country"] = "Original"

# Data wrangling with the proportions

In [10]:
country_a_counts = data.groupby("Country")[[str(i) for i in np.arange(1, 18)]].sum()

In [11]:
country_sample_sizes = data.groupby("Country")[[str(i) for i in np.arange(1, 18)]].count()
country_b_counts = country_sample_sizes - country_a_counts

In [12]:
%load_ext RWinOut

In [13]:
%R -i country_a_counts
%R -i country_sample_sizes
%R -i country_b_counts

C:\ProgramData\Anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [14]:
%%R
library(metafor)

In [15]:
%%R
rownames(country_a_counts)

 [1] "Australia"      "Austria"        "Belgium"        "Bulgaria"      
 [5] "Chile"          "Denmark"        "Germany"        "Hong Kong"     
 [9] "Hungary"        "Ireland"        "Italy"          "Mainland China"
[13] "Norway"         "Serbia"         "Slovenia"       "Spain"         
[17] "Sweden"         "UK"             "USA"           


In [16]:
%%R
fit = escalc(measure="PR", xi=country_a_counts$X1, ni=country_sample_sizes$X1, slab=rownames(country_a_counts))
summary(fit)

       yi     vi    sei      zi  ci.lb  ci.ub 
1  0.4468 0.0009 0.0296 15.0920 0.3888 0.5048 
2  0.1982 0.0014 0.0378  5.2381 0.1240 0.2724 
3  0.1927 0.0008 0.0285  6.7700 0.1369 0.2485 
4  0.1811 0.0012 0.0342  5.2997 0.1141 0.2481 
5  0.1862 0.0010 0.0323  5.7600 0.1228 0.2496 
6  0.2000 0.0011 0.0327  6.1237 0.1360 0.2640 
7  0.3333 0.0007 0.0261 12.7867 0.2822 0.3844 
8  0.2250 0.0011 0.0330  6.8155 0.1603 0.2897 
9  0.2551 0.0008 0.0280  9.1235 0.2003 0.3100 
10 0.1406 0.0005 0.0217  6.4723 0.0980 0.1832 
11 0.2658 0.0006 0.0255 10.4384 0.2159 0.3157 
12 0.1892 0.0006 0.0243  7.7739 0.1415 0.2369 
13 0.1770 0.0006 0.0254  6.9715 0.1272 0.2268 
14 0.3211 0.0009 0.0298 10.7876 0.2628 0.3795 
15 0.2475 0.0009 0.0304  8.1515 0.1880 0.3070 
16 0.2764 0.0010 0.0317  8.7182 0.2142 0.3385 
17 0.2230 0.0012 0.0353  6.3165 0.1538 0.2922 
18 0.3138 0.0007 0.0272 11.5158 0.2604 0.3672 
19 0.2510 0.0008 0.0278  9.0247 0.1965 0.3055 


### Running a random-effects log-odds model on the item-by-item response proportion. I chose log-odds because they tend to be normally distributed (which improves the validity of the model inferences) and because 0 means that people were indifferent between the options, which is the main effect we want to investigate.

In [17]:
%%R
model_list = list()
estimate_list = list()
lb_list = list()
ub_list = list()
z_list = list()
p_list = list()
I2_list = list()
q_list = list()
qp_list = list()
X = 0
for (item in c("X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10",
              "X11", "X12", "X13", "X14", "X15", "X16", "X17"))
{
X = X + 1
fit = rma.uni(measure="PLO", xi=country_a_counts[[item]], ni=country_sample_sizes[[item]],
              method="ML", slab=rownames(country_a_counts))
print(item)
print(summary(fit))
model_list[[X]] = fit
estimate_list[[X]] = fit$beta[[1]]
lb_list[[X]] = fit$ci.lb
ub_list[[X]] = fit$ci.ub
z_list[[X]] = fit$zval
p_list[[X]] = fit$pval
I2_list[[X]] = fit$I2
q_list[[X]] = fit$QM[[1]]
qp_list[[X]] = fit$QMp

}

[1] "X1"

Random-Effects Model (k = 19; tau^2 estimator: ML)

  logLik  deviance       AIC       BIC      AICc 
 -8.3505   50.0557   20.7009   22.5898   21.4509   

tau^2 (estimated amount of total heterogeneity): 0.1174 (SE = 0.0473)
tau (square root of estimated tau^2 value):      0.3426
I^2 (total heterogeneity / total variability):   82.19%
H^2 (total variability / sampling variability):  5.61

Test for Heterogeneity:
Q(df = 18) = 119.1686, p-val < .0001

Model Results:

estimate      se      zval    pval    ci.lb    ci.ub 
 -1.1452  0.0878  -13.0478  <.0001  -1.3173  -0.9732  *** 

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

[1] "X2"

Random-Effects Model (k = 19; tau^2 estimator: ML)

  logLik  deviance       AIC       BIC      AICc 
 -1.5318   41.7131    7.0636    8.9525    7.8136   

tau^2 (estimated amount of total heterogeneity): 0.0476 (SE = 0.0222)
tau (square root of estimated tau^2 value):      0.2182
I^2 (total heterogeneity / total variability): 

In [18]:
%%R
library(gtools)

### Preparing tables for assessing heterogeneity and overall difference from chance

In [19]:
%%R
z_list = as.numeric(unlist(z_list))
p_list = as.numeric(unlist(p_list))
I2_list = as.numeric(unlist(I2_list))
q_list = as.numeric(unlist(q_list))
qp_list = as.numeric(unlist(qp_list))

In [20]:
%R -o z_list
%R -o p_list
%R -o I2_list
%R -o q_list
%R -o qp_list
z_list = np.array(z_list)
p_list = np.array(p_list)
I2_list = np.array(I2_list)
q_list = np.array(q_list)
qp_list = np.array(qp_list)

In [21]:
q_list

array([1.70245661e+02, 6.04026862e+01, 4.79787135e+02, 3.82839332e-01,
       3.18764681e+02, 2.30122812e+02, 4.77784798e+02, 6.11066118e-02,
       4.38918864e+02, 2.32187725e+01, 3.97899686e+02, 2.36527559e+02,
       5.14616843e+01, 4.01523456e+02, 9.56669503e+01, 1.42445640e+01,
       1.94938233e+01])

In [22]:
het_df = pd.DataFrame([[str(x) for x in np.arange(1, 18)], q_list, qp_list, I2_list, z_list, p_list]).transpose()
het_df.columns = ["Item", "Q (df = 18)", "Q p-value", "I2", "z-statistic", "z p-value"]
het_df["Q (df = 18)"] = het_df["Q (df = 18)"].astype(float).round(2)
het_df["I2"] = het_df["I2"].astype(float).round(0).astype(int)
het_df["z-statistic"] = het_df["z-statistic"].astype(float).round(2)

het_df.to_excel("../output/Meta analysis proportions heterogeneity.xlsx", index=False)
het_df.to_csv("../output/Meta analysis proportions heterogeneity.csv", index=False)

## Transforming estimates and upper and lower confidence bounds back to proportions

In [23]:
%%R
lb_list = sapply(as.numeric(unlist(lb_list)), inv.logit)
ub_list = sapply(as.numeric(unlist(ub_list)), inv.logit)
estimate_list = sapply(as.numeric(unlist(estimate_list)), inv.logit)
p_list = as.numeric(unlist(p_list))

In [24]:
%R -o lb_list
%R -o ub_list
%R -o estimate_list
%R -o p_list
lb_list = np.array(lb_list)
ub_list = np.array(ub_list)
estimate_list = np.array(estimate_list)
p_list = np.array(p_list)

In [25]:
prop_df = pd.DataFrame([estimate_list, lb_list, ub_list, p_list]).transpose()
prop_df.columns = ["prop", "lb", "ub", "p-value"]
prop_df.head()

,prop,lb,ub,p-value
0,0.241358,0.211271,0.274240,6.539007e-39
1,0.614820,0.586543,0.642342,7.730740e-15
2,0.123918,0.106135,0.144201,2.377183e-106
3,0.509695,0.478994,0.540324,5.360876e-01
4,0.102886,0.082922,0.126991,2.691424e-71


In [26]:
prop_df["Item"] = [str(x) for x in np.arange(1, 18)]

In [27]:
prop_df["Succesful Replication"] = "Yes"
prop_df.loc[prop_df["p-value"]>.05, "Succesful Replication"] = "No"
prop_df.loc[prop_df["Item"]=="8", "Succesful Replication"] = "NA"

In [28]:
prop_df

,prop,lb,ub,p-value,Item,Succesful Replication
0,0.241358,0.211271,0.274240,6.539007e-39,1,Yes
1,0.614820,0.586543,0.642342,7.730740e-15,2,Yes
2,0.123918,0.106135,0.144201,2.377183e-106,3,Yes
3,0.509695,0.478994,0.540324,5.360876e-01,4,No
4,0.102886,0.082922,0.126991,2.691424e-71,5,Yes
5,0.653349,0.634578,0.671655,5.604791e-52,6,Yes
6,0.796322,0.775774,0.815435,6.482894e-106,7,Yes
7,0.503177,0.478004,0.528333,8.047554e-01,8,NA
8,0.795458,0.774008,0.815357,1.860989e-97,9,Yes
9,0.429331,0.401232,0.457893,1.445791e-06,10,Yes


In [29]:
prop_df.to_excel("../output/Meta analysis proportions.xlsx", index=False)
prop_df.to_csv("../output/Meta analysis proportions.csv", index=False)